In [ ]:
# 4.0 이상 버전 필요
# !pip install transformers

In [14]:
import torch
import torch.nn as nn
import torch.utils.data import dataset, DataLoader
import transformers
import glob
import os
import csv

# MELD Dataset

## Dataset Load

MELD GitHub : https://github.com/declare-lab/MELD.git

In [8]:
if not os.path.exists('MELD'):
    !git clone https://github.com/declare-lab/MELD.git

Active code page: 65001


Cloning into 'MELD'...


In [12]:
data_path = './MELD/data/MELD/*.csv'
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD\\dev_sent_emo.csv', './MELD/data/MELD\\test_sent_emo.csv', './MELD/data/MELD\\train_sent_emo.csv']


## 데이터 확인